# Predicting Codons using the trained Encoder model

In [1]:
import sys
import random
import numpy as np
import pandas as pd
import ast
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
import time

sys.path.append('../scripts')
import ml_helper
import Classifier as Classifier
import Baseline_classifiers as bc

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


### Data Preparation

In [3]:
amino_acids = ['A', 'R', 'N', 'D', 'C', 'Q', 'E', 'G', 'H', 'I', 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V', '*',
               '_']

aminoacids_to_integer = dict((a, i) for i, a in enumerate(amino_acids))
integer_to_aminoacids = dict((i, a) for i, a in enumerate(amino_acids))

codons = ['TTT', 'TTC', 'TTA', 'TTG', 'TCT', 'TCC', 'TCA', 'TCG', 'TAT', 'TAC', 'TAA', 'TAG', 'TGT', 'TGC', 'TGA',
          'TGG', 'CTT', 'CTC', 'CTA', 'CTG', 'CCT', 'CCC', 'CCA', 'CCG', 'CAT', 'CAC', 'CAA', 'CAG', 'CGT', 'CGC',
          'CGA', 'CGG', 'ATT', 'ATC', 'ATA', 'ATG', 'ACT', 'ACC', 'ACA', 'ACG', 'AAT', 'AAC', 'AAA', 'AAG', 'AGT',
          'AGC', 'AGA', 'AGG', 'GTT', 'GTC', 'GTA', 'GTG', 'GCT', 'GCC', 'GCA', 'GCG', 'GAT', 'GAC', 'GAA', 'GAG',
          'GGT', 'GGC', 'GGA', 'GGG', '___']

codons_to_integer = dict((c, i) for i, c in enumerate(codons))
integer_to_codons = dict((i, c) for i, c in enumerate(codons))

In [4]:
organism = "E.Coli"

def group_codons(sequence):
    return [''.join(sequence[i:i+3]) for i in range(0, len(sequence), 3)]

df = pd.read_pickle(f"../data/{organism}/cleanedData_test.pkl")
usage_biases = pd.read_pickle(f"../data/{organism}/usageBias.pkl")
df['codons'] = df['sequence'].apply(group_codons)

display(df.head())

,id,description,sequence,translation,seguid,codons
1,lcl|U00096.3_cds_AAC73113.1_2,lcl|U00096.3_cds_AAC73113.1_2 [gene=thrA] [loc...,"(A, T, G, C, G, A, G, T, G, T, T, G, A, A, G, ...","(M, R, V, L, K, F, G, G, T, S, V, A, N, A, E, ...",/p+3Jdgat4Fq0w2rqqay4xg8Bs4,"[ATG, CGA, GTG, TTG, AAG, TTC, GGC, GGT, ACA, ..."
5,lcl|U00096.3_cds_AAC73117.1_6,lcl|U00096.3_cds_AAC73117.1_6 [gene=yaaA] [loc...,"(A, T, G, C, T, G, A, T, T, C, T, T, A, T, T, ...","(M, L, I, L, I, S, P, A, K, T, L, D, Y, Q, S, ...",vJJ0yR31YORqwI12U79SgItYU3U,"[ATG, CTG, ATT, CTT, ATT, TCA, CCT, GCG, AAA, ..."
12,lcl|U00096.3_cds_AAC73124.1_13,lcl|U00096.3_cds_AAC73124.1_13 [gene=yaaI] [lo...,"(A, T, G, A, A, A, T, C, C, G, T, T, T, T, T, ...","(M, K, S, V, F, T, I, S, A, S, L, A, I, S, L, ...",GT2zzYZoFncaOMVxs4CEcLaePdc,"[ATG, AAA, TCC, GTT, TTT, ACG, ATT, TCC, GCC, ..."
17,lcl|U00096.3_cds_AAT48122.1_18,lcl|U00096.3_cds_AAT48122.1_18 [gene=hokC] [lo...,"(A, T, G, A, A, G, C, A, G, C, A, T, A, A, G, ...","(M, K, Q, H, K, A, M, I, V, A, L, I, V, I, C, ...",yfUY1Sxn8BgBfdGY1FQnaroApNY,"[ATG, AAG, CAG, CAT, AAG, GCG, ATG, ATT, GTC, ..."
23,lcl|U00096.3_cds_AAC73135.1_24,lcl|U00096.3_cds_AAC73135.1_24 [gene=yaaY] [lo...,"(A, T, G, T, G, C, C, G, G, C, A, C, T, C, G, ...","(M, C, R, H, S, L, R, S, D, G, A, G, F, Y, Q, ...",m/3aWNuEiWlqAe7cvTcnrZ58efA,"[ATG, TGC, CGG, CAC, TCG, TTA, CGT, AGT, GAT, ..."


In [5]:
min_length = 100
max_length = 500

df = ml_helper.filter_sequence_length(df, min_length, max_length)

In [6]:
len(df)

592

## Load trained model

In [7]:
class EncoderClassifier(nn.Module):
    def __init__(self, embed_dim, num_layers, num_heads):
        super(EncoderClassifier, self).__init__()

        self.emb = nn.Embedding(len(amino_acids), embed_dim, padding_idx=len(amino_acids)-1)
        self.encoder_layer = nn.TransformerEncoderLayer(
            d_model=embed_dim,
            nhead=num_heads,
            batch_first=True
        )
        self.encoder = nn.TransformerEncoder(
            encoder_layer=self.encoder_layer,
            num_layers=num_layers,
        )
        self.linear = nn.Linear(embed_dim, len(codons))
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = x.long()
        x = self.emb(x)
        x = self.encoder(x)
        x = self.dropout(x)
        out = self.linear(x)
        return out

In [9]:
EMBED_DIM = 256
NUM_ENCODER_LAYERS = 4
NUM_HEADS = 4

model = EncoderClassifier(
    embed_dim=EMBED_DIM,
    num_layers=NUM_ENCODER_LAYERS,
    num_heads=NUM_HEADS
).to(device)
model.load_state_dict(torch.load(f"../ml_models/{organism}/encoder_256_4_4_10e.pkl", map_location=device))

<All keys matched successfully>

## Testing the codon prediction

In [10]:
padding_pos = 'right'
padding_char = '_'
def prepare_aa_sequence(aa_sequence):
    aa_sequence = ml_helper.pad_sequence(aa_sequence, max_length, padding_pos, padding_char)
    aa_sequence = ml_helper.aa_to_int_tensor(aa_sequence, device)
    return aa_sequence

In [11]:
def predict_codons(model, aa_sequence_list):
    # Prepare data (pad, convert to tensor)
    prepared_amino_seq = []
    for seq in aa_sequence_list:
        prepared_amino_seq.append(prepare_aa_sequence(seq))

    # create data_loader for batched throughput
    batch_size = 32
    data_loader = DataLoader(prepared_amino_seq, batch_size=batch_size)

    model.eval()
    codon_predictions = []
    with torch.no_grad():
        for batch in data_loader:
            output = model(batch)  # (batch_size, seq_len, num_classes)
            for batch_i in range(output.shape[0]):
                predicted_codons = []
                for seq_i in range(output.shape[1]):
                    aa_num = batch[batch_i][seq_i].item()
                    if aa_num == aminoacids_to_integer['_']:
                        break
                    codon_idx = torch.argmax(output[batch_i][seq_i]).item()
                    codon = integer_to_codons[codon_idx]
                    predicted_codons.append(codon)
                codon_predictions.append(predicted_codons)
    return codon_predictions

In [12]:
amino_seq = df['translation'].head()
batched_predictions = predict_codons(model, amino_seq)

In [13]:
len(batched_predictions[0])

259

## Building the classifier

In [14]:
amino_acids_to_codons = {
    'A': ['GCT', 'GCC', 'GCA', 'GCG'],
    'R': ['CGT', 'CGC', 'CGA', 'CGG', 'AGA', 'AGG'],
    'N': ['AAT', 'AAC'],
    'D': ['GAT', 'GAC'],
    'C': ['TGT', 'TGC'],
    'Q': ['CAA', 'CAG'],
    'E': ['GAA', 'GAG'],
    'G': ['GGT', 'GGC', 'GGA', 'GGG'],
    'H': ['CAT', 'CAC'],
    'I': ['ATT', 'ATC', 'ATA'],
    'L': ['TTA', 'TTG', 'CTT', 'CTC', 'CTA', 'CTG'],
    'K': ['AAA', 'AAG'],
    'M': ['ATG'],
    'F': ['TTT', 'TTC'],
    'P': ['CCT', 'CCC', 'CCA', 'CCG'],
    'S': ['TCT', 'TCC', 'TCA', 'TCG', 'AGT', 'AGC'],
    'T': ['ACT', 'ACC', 'ACA', 'ACG'],
    'W': ['TGG'],
    'Y': ['TAT', 'TAC'],
    'V': ['GTT', 'GTC', 'GTA', 'GTG'],
    '*': ['TAA', 'TAG', 'TGA']
}

In [15]:
max_weighted_bc = bc.Max_Bias_Baseline_Classifier(usage_biases)
def check_and_replace_codons(aa_sequences, codon_predictions_list):
    total_codons = 0
    not_possible_codons = 0
    used_max_bias = 0
    for i, aa_seq in enumerate(aa_sequences):
        codon_predictions = codon_predictions_list[i]
        for j, aa in enumerate(aa_seq):
            total_codons += 1
            codon_pred = codon_predictions[j]
            max_bias_pred = max_weighted_bc._predict_codon(aa)
            if codon_pred not in amino_acids_to_codons[aa]:
                not_possible_codons += 1
                
                codon_predictions_list[i][j] = max_bias_pred
            else:
                if codon_pred == max_bias_pred:
                    used_max_bias += 1
    max_bias_ratio = used_max_bias / total_codons
    print(f"Model used max bias codon for {max_bias_ratio*100:.2f}% of possible codon predictions")
    not_possible_ratio = not_possible_codons / total_codons
    print(f"Replaced {not_possible_ratio*100:.2f}% of codons")
    return codon_predictions_list

In [16]:
class Encoder_Classifier(Classifier.Classifier):
    def __init__(self, trained_model, seed=42):
        self.model = trained_model
        super().__init__(seed)


    def predict_codons(self, aa_sequences, replace=False):
        predictions_list = predict_codons(self.model, aa_sequences)
        if replace:
            predictions_list = check_and_replace_codons(aa_sequences, predictions_list)
        predictions_matrix = self.pad_and_convert_seq(predictions_list)
        return predictions_matrix

In [17]:
%%time 
encoder_classifier = Encoder_Classifier(model)
pred_codons = {}
amino_seq = df['translation']
true_codons = df['codons']
#pred_codons = encoder_classifier.predict_codons(amino_seq)
pred_codons_replaced = encoder_classifier.predict_codons(amino_seq, replace=True)

Model used max bias codon for 87.43% of possible codon predictions
Replaced 0.00% of codons
CPU times: total: 2min 20s
Wall time: 37.2 s


In [18]:
accuracy = encoder_classifier.calc_accuracy(true_codons, pred_codons_replaced)
print(f"Organismus {organism} - Accuracy: {accuracy}")

Organismus E.Coli - Accuracy: 0.5129705551241068


### Compare with baseline classifier

In [19]:
max_weighted_bc = bc.Max_Bias_Baseline_Classifier(usage_biases)
amino_seq = df['translation'].apply(lambda seq: list(seq))
true_codons = df['codons']
pred_codons = max_weighted_bc.predict_codons(amino_seq)
accuracy = max_weighted_bc.calc_accuracy(true_codons, pred_codons)
print(f"Organismus {organism} - Accuracy: {accuracy}")

Organismus E.Coli - Accuracy: 0.5133919702360527
